<a href="https://colab.research.google.com/github/abhi91420/Resume_Automation/blob/main/Resume_Builder_Automate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Resume Builder Automation

- Input : Job description
- Output : Resume relavent to the job

### Input data preprocessing

we paste the job description and we extract meaningful data from it


In [1]:
job_desc= """· Java/J2EE, Spring, Rest API, Spring Boot and Micro Services - Job Description and Skillset· Experience in Java/J2EE development, Spring framework, Java persistence and API· Experience with designing and implementing Microservices based applications · Experience in designing and implementing micro services (Containers)· Strong in Integration components - Integration, RESTful web service implementations, Spring Boot, XML/XSD, JSON. MQ Integration, Junits· XML and JSON knowledge required· RAML, Swagger and WSDL knowledge a plus· Experience with setting up CI/CD environments and working within them· Experience using Agile development methodology

Good to have skills*

· Experience using Agile development methodology· Analyzes, designs, codes, tests, corrects, and documents highly complex programs to ensure optimal performance and compliance· Strong team player, excellent communication skills both verbal and written

Key responsibilities*

To understand requirements/stories as part of an Agile Team & ensure successful development as a Full Stack developer

Keywords

Java, Spring, Rest API, Spring Boot
"""
esc_char = [' ',',',':','/','\n','\t','-','$',"\\",'.']
for ch in esc_char:
  job_desc = job_desc.replace(ch,'')
job_desc_norm = job_desc.lower()

In [2]:
import pandas as pd
tech_set = pd.read_csv("https://raw.githubusercontent.com/abhi91420/Resume_Automation/main/data/list_of_softwares.csv",header = None,error_bad_lines= False)
tech_set = tech_set[0]
tech_set = tech_set.drop_duplicates()
tech_set_list= []
try:
  for i in range(tech_set.size):
    tech_set_list.append(tech_set[i])
except KeyError:
  pass
    

### Normalize the data

In [3]:
def Normalize(x):
  temp = []
  char = [' ',',',':','/','\n','\t','-','$',"\\",'.']
  for i in range(len(x)):
    x[i] = str(x[i].lower())
    for ch in char:
      x[i] = x[i].replace(ch,'')
    temp.append(x[i])
  return temp

In [4]:
tech_set_norm = Normalize(tech_set_list)

### check for technologies in the job description


In [5]:
found_set = []
for i in tech_set_norm:
  if i in job_desc_norm:
    found_set.append(i)

if len(found_set)%2 != 2:
  found_set.append("")

### Creating a document with the information
---
Required package : Python Docx 

- install command : ```!pip install python-docx```


In [6]:
!pip install python-docx

In [7]:
# custom function 

def heading(x,n):
  return resume.add_heading(x,n)

def para(x):
  return  resume.add_paragraph(x)

def center(x):
  x.alignment = WD_ALIGN_PARAGRAPH.CENTER

### Required data for document entry

- user enters his personal data (Name, No, address etc)
- prewritten set of objective paragraphs

In [9]:
import docx
from docx.enum.text import *

# initialize document object

resume = docx.Document()

In [ ]:
# User prompt information

name = input("Enter Your Name : ")
address = input("Enter your Address : ")
phone_no = input("Enter Your Contact No :")
email_id = input("Enter Email id : ")
line_2 = address+','+phone_no+','+email_id


In [9]:
# add the user prompt info to the docxfile 

title_1 = heading(name,1)
center(title_1)
title_2 = heading(line_2,2)
center(title_2)
# Line space 
para("")


In [11]:
# add objective/summary

heading("Objective Summary",2)
objective_preset = "I am a recent graduate with a master’s degree in Computer Science and have a passion for Software, Highly motivated and want to work for your reputed organization, open to learning any new technologies that are required. Skilled in technical and non-technical aspects that contribute to an efficient team and individual functionality."
objective = para(objective_preset)
para(" ")


In [ ]:
# add work experience

print("\nWork Experience Section \n")
heading("Work Experience",2)
para("")
work_title = input("Enter Your Work title : ")
company = input("Enter Company's Name : ")
heading(work_title  + "-" + company,2)
heading("Responsibilities and duties :",3)
duties = []
print("\nEnter Duties : (enter q to stop entering duties) ")
flag = ""
while flag != "q":
  flag = input("enter a duty : ")
  duties.append(". "+ flag)
temp = ""
for duty in duties[:-1]:
  temp = temp+duty+"\n"
para(temp)


In [12]:
# add education

print("\nEducation Section \n")
education = input("Enter Your Name of the Degree :")
institute = input("Name of the institute : ")
institute_location = input("City : ") +","+input("State :")
years = input("From year : ")+" - "+input("To year : ")
heading("Education",2)
para("")
heading(education,3)
para(institute+","+institute_location+"\n"+years)



Education Section 

Enter Your Name of the Degree :Master of Computer Science
Name of the institute : Lakehead University
City : Thunder Bay
State :ON
From year : 2018
To year : 202


In [10]:
# add skills

heading("Skills",2)
para("")
skill_table = resume.add_table(rows = 0,cols = 2)
count = 0
for i in range(int(len(found_set)/2)):
  table_cells = skill_table.add_row().cells    
  table_cells[0].text = found_set[count]
  count += 1
  table_cells[1].text = found_set[count]
  count += 1

In [11]:
# Save the document

resume.save("resume.docx")